In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/home/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/home/khalil.nor/Code/RMG-Py/rmgpy/test_data'}

In [3]:
thermo_libs = [
'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]


In [4]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = 'F_Abstraction',
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


In [5]:
database.kinetics.families

{'F_Abstraction': <ReactionFamily "F_Abstraction">}

In [6]:
family = database.kinetics.families["F_Abstraction"]


In [7]:
family.clean_tree()

In [8]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=100)

end = time.time()
print(end-start)

ERROR:root:234.0
ERROR:root:iter_max achieved terminating early
ERROR:root:iter_max achieved terminating early


195.49433207511902


In [9]:
len(family.groups.entries)

365

In [10]:
family.groups.entries

{'Root': <Entry index=0 label="Root">,
 'Root_1R->O': <Entry index=1 label="Root_1R->O">,
 'Root_N-1R->O': <Entry index=2 label="Root_N-1R->O">,
 'Root_1R->O_3R->O': <Entry index=3 label="Root_1R->O_3R->O">,
 'Root_1R->O_N-3R->O': <Entry index=4 label="Root_1R->O_N-3R->O">,
 'Root_N-1R->O_3R->O': <Entry index=5 label="Root_N-1R->O_3R->O">,
 'Root_N-1R->O_N-3R->O': <Entry index=6 label="Root_N-1R->O_N-3R->O">,
 'Root_1R->O_3R->O_Ext-1O-R': <Entry index=7 label="Root_1R->O_3R->O_Ext-1O-R">,
 'Root_1R->O_3R->O_Ext-3O-R_Ext-4R!H-R': <Entry index=8 label="Root_1R->O_3R->O_Ext-3O-R_Ext-4R!H-R">,
 'Root_1R->O_3R->O_1O-u0': <Entry index=9 label="Root_1R->O_3R->O_1O-u0">,
 'Root_1R->O_3R->O_N-1O-u0': <Entry index=10 label="Root_1R->O_3R->O_N-1O-u0">,
 'Root_1R->O_N-3R->O_1O-u0': <Entry index=11 label="Root_1R->O_N-3R->O_1O-u0">,
 'Root_1R->O_N-3R->O_N-1O-u0': <Entry index=12 label="Root_1R->O_N-3R->O_N-1O-u0">,
 'Root_N-1R->O_3R->O_Ext-3O-R': <Entry index=13 label="Root_N-1R->O_3R->O_Ext-3O-R">

In [11]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.17911577224731445


In [12]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

1.7777605056762695


In [13]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

1.5876598358154297


In [14]:
len(templateRxnMap)

365

In [16]:
family.clean_tree_rules()

In [17]:
start = time.time()
family.make_bm_rules_from_template_rxn_map(templateRxnMap)#,nprocs=6)
end = time.time()
print(end-start)

/home/khalil.nor/Code/RMG-Py/rmgpy/data/kinetics/family.py:3617: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for rxns, label in rxnlists])
/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


49.31952905654907


In [18]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

0.179642915725708


In [19]:
start = time.time()
errors,uncertainties = family.cross_validate(iters=0,random_state=5,folds=0,ascend=False)
end = time.time()
print(end-start)

/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


2.220993757247925


In [20]:
errors.values()

dict_values([-1.4815502647715686, -1.2633803323126445, -1.3373686466083032, 1.1173386285873483, 1.2017779612962136, 0.6127892888118722, -0.018646936895190175, 0.3903800973040277, 3.307838231001113, 0.3761562963111153, -1.7441715960234525, 0.34244087511163235, 2.7598705284078555, -1.16620428291436, 1.62918588767182, -0.411083092718251, 1.9496791293492823, 1.1903908122802627, -0.6365865832116328, 0.37457399490984766, -1.0730897807829056, -2.261685178116033, -0.7899784643609935, 2.379621371814591, 0.32191377705518004, 0.7426805828141225, 0.8692136149422504, -1.9481707042745955, -1.7170882685803965, -5.72277224428623, 1.5054477508884447, 0.21085163953238625, 1.275228730503887, 1.2334776959548654, 2.3158900493123467, 0.9942378298401978, 3.0460455712929404, 2.219495509459154, -0.9447524458152803, 5.055969497724739, 1.2761409722375303, -6.090303880959565, 0.2520876217742862, 0.4861880182779506, -0.6107292103770013, 0.17077144509535516, 0.7738372224418322, 1.084958656105513, -1.541025441484911

In [21]:
plt.figure(figsize=(10,8))
plt.hist(np.abs(list(errors.values())),bins=30, density=True)
plt.title('Decision Tree Estimator',fontsize=18)
#plt.ylim(0,.4)
plt.xlim(0,15)
plt.ylabel('Probability density',fontsize=18)
plt.xlabel(r'$|Ln(k_{est}/k_{rxn})|$',fontsize=18)

Text(0.5, 0, '$|Ln(k_{est}/k_{rxn})|$')

In [ ]:
#family.save('/Users/mattjohnson/RMGCODE/RMG-database/input/kinetics/families/')

In [22]:
save_path = os.path.join(settings['database.directory'], 'kinetics', 'families', family.name)
print(save_path)

/home/khalil.nor/Code/RMG-database/input/kinetics/families/F_Abstraction


In [23]:
family.save(save_path)

In [ ]:
#note: ran this jupyter notebook up to this point on Apr 24th, 2024 at 10:46 am 